# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [85]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Output File (CSV)
output_data_file = "../output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [86]:
vacation_df=pd.read_csv(output_data_file)
vacation_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [87]:
gmaps.configure(api_key=g_key)

In [88]:

locations = vacation_df[['Lat', 'Lng']]
humidity =vacation_df['Humidity'].astype(float)

fig = gmaps.figure(zoom_level=2,center=(15,25))
                      
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False, max_intensity = 200,
                                 point_radius = 3)
                      
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [89]:
 narrow_cities= vacation_df.loc[(vacation_df['Max Temp'] > 65) &
                         (vacation_df['Max Temp'] < 80) &
                         (vacation_df['Cloudiness'] == 0) &
                         (vacation_df['Wind Speed'] < 10)].dropna()
narrow_cities= narrow_cities.reset_index()
narrow_cities.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [90]:
hotel_df= narrow_cities
hotel_df.head()


,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17


In [91]:
hotel =[]

for index, row in hotel_df.iterrows():
    parameters_hotel= {"radius": 5000,
          "types": "hotel",
          "keyword": "hotel",
          "key": g_key,
          "location": f'{row["Lat"]}, {row["Lng"]}'}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=parameters_hotel).json()
    try:
        hotel.append(response['results'][0]['name'])
    except IndexError:
        hotel.append(np.nan)
hotel_df['Hotel Name'] = hotel
hotel_df.dropna()
hotel_df.head()



,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Pousada Paraiso Mineiro
1,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
2,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
3,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Mist Jungle Stay
4,168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17,Jægtvolden Fjordhotell


In [92]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_cities.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [93]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))